<center>
<img src="../../img/ml_theme.png">
# Дополнительное профессиональное <br> образование НИУ ВШЭ
#### Программа "Машинное обучение и майнинг данных"
<img src="../../img/faculty_logo.jpg" height="240" width="240">
## Автор материала: преподаватель Факультета Компьютерных Наук НИУ ВШЭ Кашницкий Юрий
</center>
Основано на материалах <a href="https://courses.edx.org/courses/BerkeleyX/CS100.1x/1T2015/info">курса</a> по Apache Spark 

# Занятие 9. Машинное обучение с Apache Spark
## Часть 3. Прогнозирование рейтинга фильма

![Spark Logo](http://spark-mooc.github.io/web-assets/images/ta_Spark-logo-small.png) ![Python Logo](http://spark-mooc.github.io/web-assets/images/python-logo-master-v3-TM-flattened_small.png)
#### Рекомендация фильмов пользователю. Для начала что-то простое, затем используем метод Alternating Least Squares библиотеки [Spark MLlib](https://spark.apache.org/mllib/)
#### Для демонстрации используем 500,000 рейтингов из набора [movielens](http://grouplens.org/datasets/movielens/).
#### План:
#### *Часть 0*: Основы
#### *Часть 1*: Простые рекомендации
#### *Часть 2*: Коллаборативная фильтрация
#### *Часть 3*: Проверка алгоритма "на себе"

Напишем функцию для тестирования результатов.

In [5]:
import sys
import os
# from test_helper import Test

baseDir = os.path.join('../../data/')

ratingsFilename = os.path.join(baseDir, 'ratings.dat')
moviesFilename = os.path.join(baseDir, 'movies.dat')

In [4]:
rawMovies = sc.textFile(moviesFilename)
moviesRDD = rawMovies.map(get_movie_tuple).cache()
moviesRDD.count()

3883

### **Часть 0: Основы**
#### Каждая строка файла `ratings.dat.gz` имеет вид:
####   `UserID::MovieID::Rating::Timestamp`
#### Каждая строка файла `movies.dat` имеет вид:
####   `MovieID::Title::Genres`
#### Формат поля `Genres`:
####   `Genres1|Genres2|Genres3|...`
* #### Для каждой строки файла с рейтингами создаем кортеж (UserID, MovieID, Rating).
* #### Для каждой строки файла с фильмами создаем кортеж (MovieID, Title).

In [6]:
numPartitions = 2
rawRatings = sc.textFile(ratingsFilename).repartition(numPartitions)
rawMovies = sc.textFile(moviesFilename)

def get_ratings_tuple(entry):
    """ Parse a line in the ratings dataset
    Args:
        entry (str): a line in the ratings dataset in the form of UserID::MovieID::Rating::Timestamp
    Returns:
        tuple: (UserID, MovieID, Rating)
    """
    items = entry.split('::')
    return int(items[0]), int(items[1]), float(items[2])


def get_movie_tuple(entry):
    """ Parse a line in the movies dataset
    Args:
        entry (str): a line in the movies dataset in the form of MovieID::Title::Genres
    Returns:
        tuple: (MovieID, Title)
    """
    items = entry.split('::')
    return int(items[0]), items[1]


ratingsRDD = rawRatings.map(get_ratings_tuple).cache()
moviesRDD = rawMovies.map(get_movie_tuple).cache()

ratingsCount = ratingsRDD.count()
moviesCount = moviesRDD.count()

print('There are %s ratings and %s movies in the datasets' % (ratingsCount, moviesCount))
print('Ratings: %s' % ratingsRDD.take(3))
print('Movies: %s' % moviesRDD.take(3))

There are 487650 ratings and 3883 movies in the datasets
Ratings: [(1, 1193, 5.0), (1, 914, 3.0), (1, 2355, 5.0)]
Movies: [(1, u'Toy Story (1995)'), (2, u'Jumanji (1995)'), (3, u'Grumpier Old Men (1995)')]


#### Создадим функцию для сортировки и по ключам, и по значениям.

In [7]:
def sortFunction(tuple):
    """ Construct the sort string (does not perform actual sorting)
    Args:
        tuple: (rating, MovieName)
    Returns:
        sortString: the value to sort with, 'rating MovieName'
    """
    key = unicode('%.3f' % tuple[0])
    value = tuple[1]
    return (key + ' ' + value)

tmp1 = [(1, u'alpha'), (2, u'alpha'), (2, u'beta'), (3, u'alpha'), (1, u'epsilon'), (1, u'delta')]
tmp2 = [(1, u'delta'), (2, u'alpha'), (2, u'beta'), (3, u'alpha'), (1, u'epsilon'), (1, u'alpha')]

oneRDD = sc.parallelize(tmp1)
twoRDD = sc.parallelize(tmp2)

print(oneRDD.sortBy(sortFunction, True).collect())
print(twoRDD.sortBy(sortFunction, True).collect())

[(1, u'alpha'), (1, u'delta'), (1, u'epsilon'), (2, u'alpha'), (2, u'beta'), (3, u'alpha')]
[(1, u'alpha'), (1, u'delta'), (1, u'epsilon'), (2, u'alpha'), (2, u'beta'), (3, u'alpha')]


### **Часть 1: Простые рекомендации**
#### Простой способ - рекомендовать фильмы с высоким средним рейтингом. Здесь найдем средние рейтинги 20 фильмов с более чем 500 оценками. 

#### **(1a) Средние рeйтинги фильмов**
#### `getCountsAndAverages()` принимает кортеж вида (MovieID, (Rating1, Rating2, Rating3, ...)) и возвращает кортеж вида (MovieID, (число оценок, средняя оценка))

In [8]:
def getCountsAndAverages(IDandRatingsTuple):
    """ Calculate average rating
    Args:
        IDandRatingsTuple: a single tuple of (MovieID, (Rating1, Rating2, Rating3, ...))
    Returns:
        tuple: a tuple of (MovieID, (number of ratings, averageRating))
    """
    movie_id, ratings = IDandRatingsTuple
    num_ratings = len(ratings)
    avg_rating  = float(sum(ratings)) / num_ratings
    return (movie_id, (num_ratings, avg_rating))

#### **(1b) Фильмы с высокими средними оценками**

#### Шаги:
* ####  `ratingsRDD` состоит из кортежей вида (UserID, MovieID, Rating). Создадим RDD вида (MovieID, итератор по оценкам фильма с номером MovieID).
* #### С помощью `getCountsAndAverages()` посчитаем число оценок и среднюю оценку каждого фильма:  (MovieID, (число оценок, средняя оценка)).
* #### Вместо ID фильмов хотим видеть их названия. В конце получится RDD вида  (средняя оценка, название фильма, число оценок). Например, `[(3.6818181818181817, u'Happiest Millionaire, The (1967)', 22), (3.0468227424749164, u'Grumpier Old Men (1995)', 299), (2.882978723404255, u'Hocus Pocus (1993)', 94)]`

In [10]:
# From ratingsRDD with tuples of (UserID, MovieID, Rating) create an RDD with tuples of
# the (MovieID, iterable of Ratings for that MovieID)
movieIDsWithRatingsRDD = (ratingsRDD
                          .map(lambda (user_id, movie_id, rating): (movie_id, rating))
                         .groupByKey())
print('movieIDsWithRatingsRDD: %s\n' % movieIDsWithRatingsRDD.take(3))

# Using `movieIDsWithRatingsRDD`, compute the number of ratings and average rating for each movie to
# yield tuples of the form (MovieID, (number of ratings, average rating))
movieIDsWithAvgRatingsRDD = (movieIDsWithRatingsRDD
                             .map(lambda (movie_id, rating): getCountsAndAverages((movie_id, rating))))
print('movieIDsWithAvgRatingsRDD: %s\n' % movieIDsWithAvgRatingsRDD.take(3))

# To `movieIDsWithAvgRatingsRDD`, apply RDD transformations that use `moviesRDD` to get the movie
# names for `movieIDsWithAvgRatingsRDD`, yielding tuples of the form
# (average rating, movie name, number of ratings)
movieNameWithAvgRatingsRDD = (moviesRDD
                              .join(movieIDsWithAvgRatingsRDD)
                             .map(lambda (movie_id, (movie_name, (num_ratings, avg_rating))): 
                                                    (avg_rating, movie_name, num_ratings)))
print('movieNameWithAvgRatingsRDD: %s\n' % movieNameWithAvgRatingsRDD.take(3))

movieIDsWithRatingsRDD: [(2, <pyspark.resultiterable.ResultIterable object at 0x7ff8028ffed0>), (4, <pyspark.resultiterable.ResultIterable object at 0x7ff8029bb490>), (6, <pyspark.resultiterable.ResultIterable object at 0x7ff8029bb1d0>)]

movieIDsWithAvgRatingsRDD: [(2, (332, 3.174698795180723)), (4, (71, 2.676056338028169)), (6, (442, 3.7918552036199094))]

movieNameWithAvgRatingsRDD: [(3.5671641791044775, u'Great Mouse Detective, The (1986)', 67), (3.763948497854077, u'Moonstruck (1987)', 466), (2.676056338028169, u'Waiting to Exhale (1995)', 71)]



#### **(1c) Фильмы с самыми высокими рейтингами, основанными более чем на 500 оценках **

In [18]:
# Apply an RDD transformation to `movieNameWithAvgRatingsRDD` to limit the results to movies with
# ratings from more than 500 people. We then use the `sortFunction()` helper function to sort by the
# average rating to get the movies in order of their rating (highest rating first)
movieLimitedAndSortedByRatingRDD = (movieNameWithAvgRatingsRDD
                                    .filter(lambda (avg_rating, movie_name, num_ratings):
                                           num_ratings > 500)
                                    .sortBy(sortFunction, ascending=False))
print('Movies with highest ratings: %s' % movieLimitedAndSortedByRatingRDD.take(20))

Movies with highest ratings: [(4.5349264705882355, u'Shawshank Redemption, The (1994)', 1088), (4.515798462852263, u"Schindler's List (1993)", 1171), (4.512893982808023, u'Godfather, The (1972)', 1047), (4.510460251046025, u'Raiders of the Lost Ark (1981)', 1195), (4.505415162454874, u'Usual Suspects, The (1995)', 831), (4.457256461232604, u'Rear Window (1954)', 503), (4.45468509984639, u'Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1963)', 651), (4.43953006219765, u'Star Wars: Episode IV - A New Hope (1977)', 1447), (4.4, u'Sixth Sense, The (1999)', 1110), (4.394285714285714, u'North by Northwest (1959)', 700), (4.379506641366224, u'Citizen Kane (1941)', 527), (4.375, u'Casablanca (1942)', 776), (4.363975155279503, u'Godfather: Part II, The (1974)', 805), (4.358816276202219, u"One Flew Over the Cuckoo's Nest (1975)", 811), (4.358173076923077, u'Silence of the Lambs, The (1991)', 1248), (4.335826477187734, u'Saving Private Ryan (1998)', 1337), (4.32624113475177

## **Часть 2: Коллаборативная фильтрация**
#### Основное допущение коллаборативной фильтрации: если Кирилл относится к какому-то предмету (фильму, книге) так же, как Федя,  то скорее его отношение и к другому предмету ближе ко мнению Феди, чем ко мнению случайно выбранного из толпы человека.
#### Анимация из [Wikipedia](https://en.wikipedia.org/?title=Collaborative_filtering)
![collaborative filtering](https://courses.edx.org/c4x/BerkeleyX/CS100.1x/asset/Collaborative_filtering.gif)

#### Для рекомендации фильмов начинаем  с матрицы оценок фильмов пользователями. Матрица обозначена красным цветом на картинке ниже. Каждый столбец представляет пользователя (зеленый), а каждая строка - какой-то фильм (синий).

#### Идея в том, чтобы приближенно представить эту матрицу в виде произведения двух других матриц, суть которых - некоторые признаки пользователей (зеленая матрица) и некоторые признаки фильмов (синяя матрица).
![factorization](http://spark-mooc.github.io/web-assets/images/matrix_factorization.png)
#### Матрицы хочется подобрать так, чтобы минимизировать ошибку в оценках для пар (пользователь, фильм), где оценка известна. Метод [Alternating Least Squares](http://stanford.edu/~rezab/dao/notes/lec14.pdf) делает это так: сначала одна из матриц (признаки пользователей или фильмов) заполняется случайным числами, потом значения второй матрицы подбираются решением задачи оптимизации. Потом наоборот. И так далее до сходимости. 

#### **(2a) Создание обучающей выборки**
#### Используем [randomSplit()](https://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.randomSplit) для разбиения данных на обучающую (`trainingRDD`), проверочную (`validationRDD`) и тестовую (`testRDD`) выборки.

In [22]:
trainingRDD, validationRDD, testRDD = ratingsRDD.randomSplit([6, 2, 2], seed=0L)

print('Training: %s, validation: %s, test: %s\n' % (trainingRDD.count(),
                                                    validationRDD.count(),
                                                    testRDD.count()))
print(trainingRDD.take(3))
print(validationRDD.take(3))
print(testRDD.take(3))

Training: 292716, validation: 96902, test: 98032

[(1, 914, 3.0), (1, 2355, 5.0), (1, 595, 5.0)]
[(1, 1287, 5.0), (1, 594, 4.0), (1, 1270, 5.0)]
[(1, 1193, 5.0), (1, 2398, 4.0), (1, 1035, 5.0)]


#### **(2b) Среднеквадратичная ошибка (RMSE)**
#### Пусть имеются  `predictedRDD` и `actualRDD`, каждый вида (UserID, MovieID, Rating). Тогда  $ RMSE = \sqrt{\frac{\sum_{i = 1}^{n} (x_i - y_i)^2}{n}}$, где $x_i$ и $y_i$ - оценки (Rating) из `predictedRDD` и `actualRDD` соответственно, $n$ - число троек в этих RDD.
#### Шаги:
* #### Преобразовать `predictedRDD` и `actualRDD` к виду ((UserID, MovieID), Rating)

* #### Для совпадающих пар (UserID, MovieID) посчитать квадраты разниц оценок  $ (x_i - y_i)^2$. Здесь важно не использовать метод `collect()`, потому что результаты такой операции (полученный список) могут не поместиться на одной машине. 
* #### Посчитать $$ SE = \sum_{i = 1}^{n} (x_i - y_i)^2 $$
* #### Найти *n* - число пар, по которым была посчитана сумма квадратов разниц оценок. 
* #### Посчитать RMSE

In [23]:
import math

def computeError(predictedRDD, actualRDD):
    """ Compute the root mean squared error between predicted and actual
    Args:
        predictedRDD: predicted ratings for each movie and each user where each entry is in the form
                      (UserID, MovieID, Rating)
        actualRDD: actual ratings where each entry is in the form (UserID, MovieID, Rating)
    Returns:
        RSME (float): computed RSME value
    """
    # Transform predictedRDD into the tuples of the form ((UserID, MovieID), Rating)
    predictedReformattedRDD = predictedRDD.map(lambda (u_id, m_id, r): ((u_id, m_id), r))

    # Transform actualRDD into the tuples of the form ((UserID, MovieID), Rating)
    actualReformattedRDD = actualRDD.map(lambda (u_id, m_id, r): ((u_id, m_id), r))

    # Compute the squared error for each matching entry (i.e., the same (User ID, Movie ID) in each
    # RDD) in the reformatted RDDs using RDD transformtions - do not use collect()
    squaredErrorsRDD = (predictedReformattedRDD
                        .join(actualReformattedRDD)
                        .map(lambda ((u_id, m_id), (r1, r2)): pow(r1 - r2,2))
                        )

    # Compute the total squared error - do not use collect()
    totalError = squaredErrorsRDD.sum()
    # Count the number of entries for which you computed the total squared error
    numRatings = squaredErrorsRDD.count()

    # Using the total squared error and the number of entries, compute the RSME
    return math.sqrt(float(totalError) / numRatings)

# sc.parallelize turns a Python list into a Spark RDD.
testPredicted = sc.parallelize([
    (1, 1, 5),
    (1, 2, 3),
    (1, 3, 4),
    (2, 1, 3),
    (2, 2, 2),
    (2, 3, 4)])
testActual = sc.parallelize([
     (1, 2, 3),
     (1, 3, 5),
     (2, 1, 5),
     (2, 2, 1)])
testPredicted2 = sc.parallelize([
     (2, 2, 5),
     (1, 2, 5)])
testError = computeError(testPredicted, testActual)
print('Error for test dataset (should be 1.22474487139): %s' % testError)

testError2 = computeError(testPredicted2, testActual)
print('Error for test dataset2 (should be 3.16227766017): %s' % testError2)

testError3 = computeError(testActual, testActual)
print('Error for testActual dataset (should be 0.0): %s' % testError3)

Error for test dataset (should be 1.22474487139): 1.22474487139
Error for test dataset2 (should be 3.16227766017): 3.16227766017
Error for testActual dataset (should be 0.0): 0.0


#### **(2c) Использование ALS.train()**
#### Шаги:
* #### Выбор параметров модели. Самый важный параметр метода `ALS.train()` - *rank*, число строк в матрице пользователей или, что то же самое, число столбцов в матрице фильмов. (Обычно малый ранг приводит к недообучению, а большой - к переобучению).  Будем для обучающей выборки `trainingRDD` использовать ранги 4, 8 и 12. 
* #### Создаем модель `ALS.train(trainingRDD, rank, seed=seed, iterations=iterations, lambda_=regularizationParameter)` со следующими параметрами: RDD? составленный из троек (UserID, MovieID, rating)  - обучающая выборка, ранг матриц разложения (4, 8 и 12), число итераций алгоритма и коэффициент регуляризации (будем использовать `regularizationParameter`=0.1).
* #### Для прогноза используем `validationForPredictRDD`, составленный из пар (UserID, MovieID)  `validationRDD`.
* #### Вычисление ошибки прогнозна рейтингов для `validationForPredictRDD` (метод [model.predictAll()](https://spark.apache.org/docs/latest/api/python/pyspark.mllib.html#pyspark.mllib.recommendation.MatrixFactorizationModel.predictAll)) с реальными с помощью написанной ранее функции `computeError()`.

In [24]:
from pyspark.mllib.recommendation import ALS

validationForPredictRDD = validationRDD.map(lambda (u_id, m_id, r): 
                                            (u_id, m_id))

seed = 5L
iterations = 5
regularizationParameter = 0.1
ranks = [4, 8, 12]
errors = [0, 0, 0]
err = 0
tolerance = 0.02

minError = float('inf')
bestRank = -1
bestIteration = -1
for rank in ranks:
    model = ALS.train(trainingRDD, rank, seed=seed, iterations=iterations,
                      lambda_=regularizationParameter)
    predictedRatingsRDD = model.predictAll(validationForPredictRDD)
    error = computeError(predictedRatingsRDD, validationRDD)
    errors[err] = error
    err += 1
    print('For rank %s the RMSE is %s' % (rank, error))
    if error < minError:
        minError = error
        bestRank = rank

print('The best model was trained with rank %s' % bestRank)

For rank 4 the RMSE is 0.895405660311
For rank 8 the RMSE is 0.895514822303
For rank 12 the RMSE is 0.894980442967
The best model was trained with rank 12


#### **(2d) Проверка модели**
#### Шаги:
* #### Обучение модели на выборке `trainingRDD`  с рангом  `bestRank` и прочими.
* #### Прогнозирование для пар (UserID, MovieID) из  `testForPredictingRDD`.
* #### Вычисление ошибки для  `testRDD` с помощью функции `computeError`.


In [25]:
myModel = ALS.train(trainingRDD, bestRank, seed=seed, iterations=iterations,
                    lambda_=regularizationParameter)
testForPredictingRDD = testRDD.map(lambda (u_id, m_id, r): (u_id, m_id))
predictedTestRDD = myModel.predictAll(testForPredictingRDD)

testRMSE = computeError(testRDD, predictedTestRDD)

print('The model had a RMSE on the test set of %s' % testRMSE)

The model had a RMSE on the test set of 0.896040796967


#### **(2e) Сравнение с предсказанием по среднему**
#### Шаги:
* #### Подсчет среднего рейтинга по каждому фильму для `trainingRDD`.
* #### Создание троек (userID, movieID, average rating) для  `testRDD`.
* #### Оценка с помощью `computeError`

In [26]:
trainingAvgRating = trainingRDD.map(lambda (u_id, m_id, r): r).mean()
print('The average rating for movies in the training set is %s' 
      % trainingAvgRating)

testForAvgRDD = testRDD.map(lambda (u_id, m_id, r): 
                            (u_id, m_id, trainingAvgRating))
testAvgRMSE = computeError(testRDD, testForAvgRDD)
print('The RMSE on the average set is %s' % testAvgRMSE)

The average rating for movies in the training set is 3.57409571052
The RMSE on the average set is 1.12036693569


#### Теперь все готово для предсказания рейтинга фильмов!

## **Часть 3. Проверка алгоритма "на себе"**

#### **(3a) Добавление собственных оценок в `ratingsRDD`**
#### В помощь - рейтинг самых популярных фильмов

In [27]:
print('Most rated movies:')
print('(average rating, movie name, number of reviews)')
for ratingsTuple in movieLimitedAndSortedByRatingRDD.take(50):
    print(ratingsTuple)

Most rated movies:
(average rating, movie name, number of reviews)
(4.5349264705882355, u'Shawshank Redemption, The (1994)', 1088)
(4.515798462852263, u"Schindler's List (1993)", 1171)
(4.512893982808023, u'Godfather, The (1972)', 1047)
(4.510460251046025, u'Raiders of the Lost Ark (1981)', 1195)
(4.505415162454874, u'Usual Suspects, The (1995)', 831)
(4.457256461232604, u'Rear Window (1954)', 503)
(4.45468509984639, u'Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1963)', 651)
(4.43953006219765, u'Star Wars: Episode IV - A New Hope (1977)', 1447)
(4.4, u'Sixth Sense, The (1999)', 1110)
(4.394285714285714, u'North by Northwest (1959)', 700)
(4.379506641366224, u'Citizen Kane (1941)', 527)
(4.375, u'Casablanca (1942)', 776)
(4.363975155279503, u'Godfather: Part II, The (1974)', 805)
(4.358816276202219, u"One Flew Over the Cuckoo's Nest (1975)", 811)
(4.358173076923077, u'Silence of the Lambs, The (1991)', 1248)
(4.335826477187734, u'Saving Private Ryan (1998)', 13

####  ID пользователя 0 еще не было. Используем его. Создадим свой список с оценками фильмов - тройки вида  `(myUserID, movieID, оценка)`.

In [28]:
myUserID = 0

# Note that the movie IDs are the *last* number on each line. A common error was to use the number of ratings as the movie ID.
myRatedMovies = [
     (myUserID, 318, 5),
    (myUserID, 858, 3),
    (myUserID, 260, 2),
    (myUserID, 912, 5),
    (myUserID, 2028, 3),
    (myUserID, 2571, 3),
    (myUserID, 2324, 5),
    (myUserID, 292, 1),
    (myUserID, 1225, 5),
    (myUserID, 1617, 2),
    (myUserID, 111, 5),
    (myUserID, 1704, 5),
    (myUserID, 1240, 3),
    (myUserID, 1387, 2)
     # The format of each line is (myUserID, movie ID, your rating)
     # For example, to give the movie "Star Wars: Episode IV - A New Hope (1977)" a five rating, you would add the following line:
     #   (myUserID, 260, 5),
    ]

myRatingsRDD = sc.parallelize(myRatedMovies)
print('My movie ratings: %s' % myRatingsRDD.take(10))

My movie ratings: [(0, 318, 5), (0, 858, 3), (0, 260, 2), (0, 912, 5), (0, 2028, 3), (0, 2571, 3), (0, 2324, 5), (0, 292, 1), (0, 1225, 5), (0, 1617, 2)]


#### **(3b) Добавление собственных оценок в обучающую выборку**
#### Для этого используем преобразование [union()](http://spark.apache.org/docs/latest/api/python/pyspark.rdd.RDD-class.html#union).

In [29]:
trainingWithMyRatingsRDD = trainingRDD.union(myRatingsRDD)

print('The training dataset now has %s more entries than the original training dataset' %
       (trainingWithMyRatingsRDD.count() - trainingRDD.count()))
assert (trainingWithMyRatingsRDD.count() - trainingRDD.count()) == myRatingsRDD.count()

The training dataset now has 14 more entries than the original training dataset


#### **(3c) Обучение модели с добавленными оценками**

In [30]:
myRatingsModel = ALS.train(trainingWithMyRatingsRDD, bestRank, 
                           seed=seed, iterations=iterations,
                           lambda_=regularizationParameter)

#### **(3d) Подсчет RMSE для новой модели**

In [31]:
predictedTestMyRatingsRDD = myRatingsModel.predictAll(testForPredictingRDD)
testRMSEMyRatings = computeError(testRDD, predictedTestMyRatingsRDD)
print('The model had a RMSE on the test set of %s' % testRMSEMyRatings)

The model had a RMSE on the test set of 0.895986406404


#### **(3e) Предсказание оценок**
#### Шаги:
* #### Используя список `myRatedMovies`, преобразуем `moviesRDD` к виду (myUserID, Movie ID) с фильмами, не оцененными пользователем.
* #### Для прогноза используем `myUnratedMoviesRDD` и  myRatingsModel.predictAll().

In [32]:
myRatedMoviesRDD = myRatingsRDD.map(lambda (m_id, m_name, r): (m_id, m_name))

myUnratedMoviesRDD = (moviesRDD
                      .map(lambda (m_id, m_name): (myUserID, m_id))
                     .subtract(myRatedMoviesRDD))

# Use the input RDD, myUnratedMoviesRDD, with myRatingsModel.predictAll() to predict your ratings for the movies
predictedRatingsRDD = myRatingsModel.predictAll(myUnratedMoviesRDD)
print(predictedRatingsRDD.take(5))

[Rating(user=0, product=3586, rating=1.9539098104725605), Rating(user=0, product=1084, rating=3.372458509086949), Rating(user=0, product=1410, rating=2.6245710967990514), Rating(user=0, product=3456, rating=3.1959902790419363), Rating(user=0, product=3702, rating=2.5641995134422624)]


#### **(3f) Рекомендация фильмов**
#### Отберем 25 фильмов с лучшими предсказанными оценками.
#### Шаги:
* #### Отбор фильмов в большим числом оценок (например, больше 75)
* #### Преобразуем `predictedRatingsRDD` в RDD вида (Movie ID, Predicted Rating): `[(3456, -0.5501005376936687), (1080, 1.5885892024487962), (320, -3.7952255522487865)]`
* #### С помощью `predictedRDD` и `movieCountsRDD` получим RDD вида (Movie ID, (предсказанный рейтинг, число оценок)): `[(2050, (0.6694097486155939, 44)), (10, (5.29762541533513, 418)), (2060, (0.5055259373841172, 97))]`
* #### С помощью `predictedWithCountsRDD` и `moviesRDD` получим RDD вида (предсказанный рейтинг, название фильма, число оценок) для фильмов с более чем 75 оценками. Например: `[(7.983121900375243, u'Under Siege (1992)'), (7.9769201864261285, u'Fifth Element, The (1997)')]`

In [33]:
# Transform movieIDsWithAvgRatingsRDD from part (1b), which has the form (MovieID, (number of ratings, average rating)), 
# into and RDD of the form (MovieID, number of ratings)
movieCountsRDD = movieIDsWithAvgRatingsRDD.map(lambda (m_id, (num_r, avg_r)): 
                                               (m_id, num_r))

# Transform predictedRatingsRDD into an RDD with entries that are pairs of the form (Movie ID, Predicted Rating)
predictedRDD = predictedRatingsRDD.map(lambda rating: (rating.product, 
                                                       rating.rating)) 

# Use RDD transformations with predictedRDD and movieCountsRDD to yield an RDD with tuples of the form 
# (Movie ID, (Predicted Rating, number of ratings))
predictedWithCountsRDD  = (predictedRDD
                           .join(movieCountsRDD))


#Use RDD transformations with PredictedWithCountsRDD and moviesRDD to yield an RDD with tuples of the form 
#(Predicted Rating, Movie Name, number of ratings), for movies with more than 75 ratings
ratingsWithNamesRDD = (moviesRDD
                       .join(predictedWithCountsRDD.filter(lambda (id, (rat, num_r)): num_r > 75))
                       .map(lambda (movie_id, 
                                    (movie_name, (num_ratings, avg_rating))): 
                                    (avg_rating, movie_name, num_ratings)))

predictedHighestRatedMovies = ratingsWithNamesRDD.takeOrdered(25, 
                                                              key=lambda x: -x[2])
print('My highest rated movies as predicted (for movies with more than 75 reviews):\n%s' %
        '\n'.join(map(str, predictedHighestRatedMovies)))

My highest rated movies as predicted (for movies with more than 75 reviews):
(171, u'Mr. Smith Goes to Washington (1939)', 4.507688210898011)
(170, u"Sophie's Choice (1982)", 4.359257688882978)
(100, u'Wings of Desire (Der Himmel \ufffdber Berlin) (1987)', 4.325775731314393)
(451, u'To Kill a Mockingbird (1962)', 4.251001290459945)
(811, u"One Flew Over the Cuckoo's Nest (1975)", 4.229691012313029)
(129, u'City Lights (1931)', 4.224421324060318)
(306, u'Harold and Maude (1971)', 4.221661171431236)
(124, u'Bicycle Thief, The (Ladri di biciclette) (1948)', 4.2067631775228005)
(246, u'Ordinary People (1980)', 4.184303651854811)
(188, u'Smoke Signals (1998)', 4.160401980982678)
(103, u'Kolya (1996)', 4.136599131505147)
(254, u'On the Waterfront (1954)', 4.117229509486224)
(87, u'400 Blows, The (Les Quatre cents coups) (1959)', 4.104632319196774)
(184, u'Breaker Morant (1980)', 4.075305851663523)
(373, u'Do the Right Thing (1989)', 4.044654556356583)
(229, u'Postino, Il (The Postman) (1994)